In [1]:
import torch
import torch.nn as nn
import polars as pl
from collections import defaultdict

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [3]:
seq2seq_BATCH_SIZE = 64
seq2seq_EMB_DIM = 128
seq2seq_HID_DIM = 258
seq2seq_N_LAYERS = 2
seq2seq_DROPOUT = 0.5
seq2seq_LEARNING_RATE = 0.001
seq2seq_EPOCHS = 10
seq2seq_FRAC = 0.2
seq2seq_SEED = 420

In [4]:
transformer_BATCH_SIZE = 32
transformer_EMB_DIM = 128
transformer_HID_DIM = 258
transformer_N_LAYERS = 2
transformer_N_HEADS = 8
transformer_FF_DIM = 512
transformer_DROPOUT = 0.1
transformer_LEARNING_RATE = 0.001
transformer_EPOCHS = 10
transformer_FRAC = 0.5
transformer_SEED = 420

In [5]:
kaggle_path= "/kaggle/input/it-en-translation/processed.parquet"
loaded_data = pl.read_parquet(kaggle_path)
print("Data loaded successfully")
loaded_data.head()

Data loaded successfully


it,en
list[str],list[str]
"[""grazie"", ""amico""]","[""thank"", ""buddy""]"
"[""di il""]","[""say""]"
"[""trifosfare"", ""sodio"", … ""sodio""]","[""sodium"", ""triphosphate"", … ""tripolyphosphate""]"
"[""invero"", ""avidare"", … ""ricchezzo""]","[""surely"", ""ardent"", … ""wealth""]"
"[""allegare""]","[""annex""]"


In [6]:
transformer_input_data = loaded_data.sample(fraction = transformer_FRAC, seed = transformer_SEED)

transformer_input_data.describe()

statistic,it,en
str,f64,f64
"""count""",480021.0,480021.0
"""null_count""",0.0,0.0
"""mean""",null,null
"""std""",null,null
"""min""",null,null
"""25%""",null,null
"""50%""",null,null
"""75%""",null,null
"""max""",null,null


In [7]:
seq2seq_input_data = loaded_data.sample(fraction = seq2seq_FRAC, seed = seq2seq_SEED)

seq2seq_input_data.describe()

statistic,it,en
str,f64,f64
"""count""",192008.0,192008.0
"""null_count""",0.0,0.0
"""mean""",null,null
"""std""",null,null
"""min""",null,null
"""25%""",null,null
"""50%""",null,null
"""75%""",null,null
"""max""",null,null


In [8]:
def build_vocab(tokens):
    vocab = defaultdict(lambda: len(vocab))  # Assign unique indices
    vocab["<pad>"]  # Reserve 0 for padding
    vocab["<unk>"]
    for token_list in tokens:
        for token in token_list:
            _ = vocab[token]
    return dict(vocab)

def preprocess_data(df):
    it_vocab = build_vocab(df["it"])
    en_vocab = build_vocab(df["en"])
    
    it_indices = [
        torch.tensor([it_vocab[token] for token in tokens], dtype=torch.long)
        for tokens in df["it"]
    ]
    en_indices = [
        torch.tensor([en_vocab[token] for token in tokens], dtype=torch.long)
        for tokens in df["en"]
    ]
    return list(zip(en_indices, it_indices)), len(it_vocab), len(en_vocab), en_vocab, it_vocab

In [9]:
seq2_seq_data_pairs, seq2seq_IT_VOCAB_SIZE, seq2seq_EN_VOCAB_SIZE, seq2seq_en_v, seq2seq_it_v = preprocess_data(seq2seq_input_data)

In [10]:
transformer_data_pairs, transformer_IT_VOCAB_SIZE, transformer_EN_VOCAB_SIZE, transformer_en_v, transformer_it_v = preprocess_data(transformer_input_data)

In [11]:
class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, hid_dim, n_layers, dropout):
        super().__init__()
        self.embedding = nn.Embedding(input_dim, emb_dim)
        self.rnn = nn.LSTM(emb_dim, hid_dim, n_layers, dropout=dropout, batch_first=True)
        self.dropout = nn.Dropout(dropout)

    def forward(self, src):
        embedded = self.dropout(self.embedding(src))
        outputs, (hidden, cell) = self.rnn(embedded)
        return hidden, cell

class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, hid_dim, n_layers, dropout):
        super().__init__()
        self.embedding = nn.Embedding(output_dim, emb_dim)
        self.rnn = nn.LSTM(emb_dim, hid_dim, n_layers, dropout=dropout, batch_first=True)
        self.fc_out = nn.Linear(hid_dim, output_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, trg, hidden, cell):
        trg = trg.unsqueeze(1)
        embedded = self.dropout(self.embedding(trg))
        output, (hidden, cell) = self.rnn(embedded, (hidden, cell))
        prediction = self.fc_out(output.squeeze(1))
        return prediction, hidden, cell

In [12]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.device = device

    def forward(self, src, trg, teacher_forcing_ratio=0.5):
        batch_size = src.size(0)
        trg_len = trg.size(1)
        trg_vocab_size = self.decoder.fc_out.out_features

        outputs = torch.zeros(batch_size, trg_len, trg_vocab_size).to(self.device)

        hidden, cell = self.encoder(src)

        trg_input = trg[:, 0]
        for t in range(1, trg_len):
            output, hidden, cell = self.decoder(trg_input, hidden, cell)
            outputs[:, t, :] = output
            top1 = output.argmax(1)
            trg_input = trg[:, t] if torch.rand(1).item() < teacher_forcing_ratio else top1

        return outputs

In [13]:
seq2seq_encoder = Encoder(seq2seq_EN_VOCAB_SIZE, seq2seq_EMB_DIM, seq2seq_HID_DIM, seq2seq_N_LAYERS, seq2seq_DROPOUT)
seq2seq_decoder = Decoder(seq2seq_IT_VOCAB_SIZE, seq2seq_EMB_DIM, seq2seq_HID_DIM, seq2seq_N_LAYERS, seq2seq_DROPOUT)
seq2seq_model = Seq2Seq(seq2seq_encoder, seq2seq_decoder, device).to(device)

seq2seq_model.load_state_dict(torch.load("/kaggle/input/dec-enc-test/seq2seq_translation_model.pth", weights_only=True))

RuntimeError: Error(s) in loading state_dict for Seq2Seq:
	size mismatch for encoder.embedding.weight: copying a param with shape torch.Size([52520, 128]) from checkpoint, the shape in current model is torch.Size([52521, 128]).
	size mismatch for decoder.embedding.weight: copying a param with shape torch.Size([79086, 128]) from checkpoint, the shape in current model is torch.Size([79087, 128]).
	size mismatch for decoder.fc_out.weight: copying a param with shape torch.Size([79086, 258]) from checkpoint, the shape in current model is torch.Size([79087, 258]).
	size mismatch for decoder.fc_out.bias: copying a param with shape torch.Size([79086]) from checkpoint, the shape in current model is torch.Size([79087]).

In [ ]:
class Transformer(nn.Module):
    def __init__(self, src_vocab_size, trg_vocab_size, emb_dim, n_heads, ff_dim, n_layers, dropout):
        super().__init__()
        # Ensure src_vocab_size matches the vocabulary size from preprocessing
        self.src_embedding = nn.Embedding(src_vocab_size, emb_dim)
        # Ensure trg_vocab_size matches the vocabulary size from preprocessing
        self.trg_embedding = nn.Embedding(trg_vocab_size, emb_dim)
        # Check that the positional encoding length covers all sequence lengths
        self.positional_encoding = self._get_positional_encoding(emb_dim)

        self.transformer = nn.Transformer(
            d_model=emb_dim,
            nhead=n_heads,
            num_encoder_layers=n_layers,
            num_decoder_layers=n_layers,
            dim_feedforward=ff_dim,
            dropout=dropout,
            batch_first=True
        )
        self.fc_out = nn.Linear(emb_dim, trg_vocab_size)

    def forward(self, src, trg):
        src_seq_len = src.size(1)  # Sequence length of source
        src = self.src_embedding(src) + self.positional_encoding[:, :src_seq_len, :].to(src.device)
        trg_seq_len = trg.size(1)  # Sequence length of target
        trg = self.trg_embedding(trg) + self.positional_encoding[:, :trg_seq_len, :].to(trg.device)

        src_mask = self._generate_square_subsequent_mask(src_seq_len).to(src.device)
        trg_mask = self._generate_square_subsequent_mask(trg_seq_len).to(trg.device)
        memory_mask = None

        # Ensure src, trg, and masks are aligned
        output = self.transformer(
            src, trg, src_mask=src_mask, tgt_mask=trg_mask, memory_mask=memory_mask
        )
        output = self.fc_out(output)

        return output

    def _generate_square_subsequent_mask(self, sz):
        return torch.triu(torch.full((sz, sz), float('-inf')), diagonal=1)

    def _get_positional_encoding(self, emb_dim, max_len=5000):
        pe = torch.zeros(1, max_len, emb_dim)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, emb_dim, 2).float() * -(torch.log(torch.tensor(10000.0)) / emb_dim))
        pe[0, :, 0::2] = torch.sin(position * div_term)
        pe[0, :, 1::2] = torch.cos(position * div_term)
        return pe

In [ ]:
transformer_model = Transformer(transformer_EN_VOCAB_SIZE, transformer_IT_VOCAB_SIZE, transformer_EMB_DIM, transformer_N_HEADS, transformer_FF_DIM, transformer_N_LAYERS, transformer_DROPOUT).to(device)

transformer_model.load_state_dict(torch.load("/kaggle/input/transformer-translator/transformer_translation_model.pth", weights_only=True))

In [ ]:
seq2seq_it_v_inv = {v: k for k, v in seq2seq_it_v.items()}

In [ ]:
transformer_it_v_inv = {v: k for k, v in transformer_it_v.items()}

In [ ]:
def tokenize(sentence, vocab):
    tokens = sentence.lower().split()  # Basic tokenization; adjust if needed
    return torch.tensor([vocab.get(token, vocab["<unk>"]) for token in tokens], dtype=torch.long)

In [ ]:
def seq2seq_translate(sentence, model, en_vocab, it_vocab_inv, device="cuda"):
    model.eval()
    model.to(device)

    input_tensor = tokenize(sentence, en_vocab).unsqueeze(0).to(device)
    hidden_state, cell_state = model.encoder(input_tensor)
    
    decoded_tokens = []
    decoder_input = torch.tensor([it_vocab["<bos>"]], dtype=torch.long, device=device)

    for _ in range(50):  # Limit translation to 50 tokens
        output, (hidden_state, cell_state) = model.decoder(
            decoder_input.unsqueeze(0), (hidden_state, cell_state)
        )
        next_token = output.argmax(2).item()
        if next_token == it_vocab["<eos>"]:
            break
        decoded_tokens.append(it_vocab_inv[next_token])
        decoder_input = torch.tensor([next_token], dtype=torch.long, device=device)

    return " ".join(decoded_tokens)

In [ ]:
def transformer_translate(sentence, model, en_vocab, it_vocab_inv, device="cuda"):
    model.eval()
    model.to(device)

    input_tensor = tokenize(sentence, en_vocab).unsqueeze(0).to(device)
    src_mask = model.generate_square_subsequent_mask(input_tensor.size(1)).to(device)

    memory = model.encoder(input_tensor, src_key_padding_mask=None)
    decoded_tokens = []
    decoder_input = torch.tensor([it_vocab["<bos>"]], dtype=torch.long, device=device)

    for _ in range(50):
        tgt_mask = model.generate_square_subsequent_mask(len(decoded_tokens) + 1).to(device)
        output = model.decoder(
            decoder_input.unsqueeze(0), memory, tgt_mask=tgt_mask
        )
        next_token = output.argmax(2).item()
        if next_token == it_vocab["<eos>"]:
            break
        decoded_tokens.append(it_vocab_inv[next_token])
        decoder_input = torch.tensor([next_token], dtype=torch.long, device=device)

    return " ".join(decoded_tokens)

In [ ]:
def translate(sentence, model_type="seq2seq", device="cuda"):
    if model_type == "seq2seq":
        return seq2seq_translate(sentence, seq2seq_model, seq2seq_en_v, seq2seq_it_v_inv, device)
    elif model_type == "transformer":
        return transformer_translate(sentence, transformer_model, transformer_en_v, transformer_it_v_inv, device)
    else:
        raise ValueError("Invalid model_type. Choose 'seq2seq' or 'transformer'.")

In [ ]:
sentence = "How are you today? good day I hope"

In [ ]:
seq2seq_translation = translate(sentence)

In [ ]:
print("Sentence Translation (Using Seq2Seq Model): ", seq2seq_translation)

In [ ]:
transformer_translation = translate(sentence, model_type = "transformer")

In [ ]:
print("Sentence Translation (Using Transformer Model): ", transformer_translation)